In [ ]:
import uproot
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import scipy 

simpath = uproot.open("/Users/juliogutierrez/Downloads/Burcu_assignments/Assignment_6/shms_8deg_9p2.root:h1411")
LD2path = uproot.open("/Volumes/T7/ROOTfiles/SHMS_replays/SHMS_skim/SHMS_17566_skim.root:T")
dummypath = uproot.open("/Volumes/T7/ROOTfiles/SHMS_replays/SHMS_skim/SHMS_17584_skim.root:T")



hsdelta = simpath["psdelta"].array(library="np")
stop_id = simpath["stop_id"].array(library="np")
hsytar = simpath["psytar"].array(library="np")
hsxptar = simpath["psxptar"].array(library="np")
hsyptar = simpath["psyptar"].array(library="np")
hsxpfp = simpath["psxpfp"].array(library="np")
hsypfp = simpath["psypfp"].array(library="np")

dumdp = dummypath["P_gtr_dp"].array(library="np")
dumxtar = dummypath["P_gtr_x"].array(library="np")
dumytar = dummypath["P_gtr_y"].array(library="np")
dumth = dummypath["P_gtr_th"].array(library="np")
dumph = dummypath["P_gtr_ph"].array(library="np")
dumCal = dummypath["P_cal_etottracknorm"].array(library="np")
dumxptar = np.tan(dumth)
dumyptar = np.tan(dumph)


ptardp = LD2path["P_gtr_dp"].array(library="np")
xtar = LD2path["P_gtr_x"].array(library="np")
ytar = LD2path["P_gtr_y"].array(library="np")
ptarth = LD2path["P_gtr_th"].array(library="np")
ptarph = LD2path["P_gtr_ph"].array(library="np")
shmsCal = LD2path["P_cal_etottracknorm"].array(library="np")
xptar = np.tan(ptarth)
yptar = np.tan(ptarph)

Ebeam = 10.54
p_spec = 9.2
delta_lo = -20
delta_hi = 20
e_min = p_spec * (1 + (0.01 * delta_lo))
e_max = p_spec * (1 + (0.01 * delta_hi))
phi_lo, theta_lo = -100, -100
phi_hi, theta_hi = 100, 100
A = 2
N_A = 6.022E23
Q_e = 1.602E-19
density = 0.16743 # g/cm^3
length = 10 #cm
thickness = density * length
sim_charge = 1 # µC
# acc_charge = 90375.561
th_central = np.deg2rad(8.0) # rad 
bin_number = 100

deltaP = e_max- e_min
domega = (phi_hi - phi_lo)*(theta_hi - theta_lo) / (1000 * 1000)
lumin = (thickness) * (sim_charge / A) * (N_A / Q_e) * 1e-39
nentries = len(hsdelta)

cuts_data = (ptardp > -10) & (ptardp < 22) & (shmsCal > 0.7) & (-0.03 < yptar) & (yptar < 0.03) & (-0.08 < xptar) & (xptar < 0.08)
cuts_dum = (dumdp > -10) & (dumdp < 22) & (dumCal > 0.7) & (dumyptar > -0.03) & (dumyptar < 0.03)  & (dumxptar > -0.08) & (dumxptar < 0.08)
cuts_sim = (hsdelta > -10) & (hsdelta <22) & (stop_id==0) 


LD_ps = 65
LD_charge = 20392.972 #µC
LD_eff =  0.99779 * 0.9727
LD_thic = 0.16743 * 10 #thickness =density * length (in g/cm^3)

dum_ps = 17
dum_charge = 21674.689 #µC
dum_eff = 0.988166 * 0.9898
dum_thic = 2.7 * 10 

R = LD_thic / dum_thic

df = pd.read_csv("/Users/juliogutierrez/Downloads/Burcu_assignments/Assignment_6/fa22_8deg_d2.csv")
Eprime, theta = np.empty(len(hsdelta)), np.empty(len(hsdelta))

sig_rad = df["cross_section"].to_numpy();

orig_parameters = df[["Eprime", "theta"]].to_numpy()

interp = LinearNDInterpolator(orig_parameters, sig_rad)

cross_section, mc_scale = np.empty_like(hsdelta), np.empty_like(hsdelta)


for i in range(len(hsdelta)):
    Eprime[i] = p_spec * (1 + (0.01 * hsdelta[i]))
    theta[i] = np.rad2deg(np.arccos((np.cos(th_central) - hsyptar[i] * np.sin(th_central)) / np.sqrt(1 + hsxptar[i]**2 + hsyptar[i]**2)))
    cross_section[i] = interp(Eprime[i], theta[i])
    mc_scale[i] =  (cross_section[i] * lumin * domega * deltaP) / (sim_charge * nentries)

dum_scale, LD_scale = np.empty_like(dumdp[cuts_dum]), np.empty_like(ptardp[cuts_data])

for i in range(len(dumdp[cuts_dum])):
    dum_scale[i] = R * dum_ps / (dum_charge * dum_eff)

for i in range(len(ptardp[cuts_data])):
    LD_scale[i] = LD_ps / (LD_charge * LD_eff)


bin_counts_dumdp, bin_edges_dumdp = np.histogram(dumdp[cuts_dum],weights=dum_scale,bins=bin_number, range=(-10,22))
bin_centers_dumdp = (bin_edges_dumdp[1:] + bin_edges_dumdp[:-1])/2

bin_counts_datadp, bin_edges_datadp = np.histogram(ptardp[cuts_data], weights=LD_scale, bins=bin_number, range=(-10,22))
bin_centers_datadp = (bin_edges_datadp[1:] + bin_edges_datadp[:-1])/2

bin_counts_simdp, bin_edges_simdp = np.histogram(hsdelta[cuts_sim], weights=mc_scale[cuts_sim], bins=bin_number, range=(-10,22))
bin_centers_simdp = (bin_edges_simdp[1:] + bin_edges_simdp[:-1])/2

bin_counts_subdp = bin_counts_datadp - R * bin_counts_dumdp
bin_edges_subdp = bin_edges_datadp - R * bin_edges_dumdp
bin_centers_subdp = (bin_edges_subdp[1:] + bin_edges_subdp[:-1])/2


# Calculate errors

def SumW2(bin_counts, hist, weights, xmin, xmax, bin_num):
    bins = np.linspace(xmin, xmax, bin_num)
    sum_of_weights = np.zeros_like(bin_counts)
    sum_of_weights_squared = np.zeros_like(bin_counts)


    for i in range(len(hist)):
        bin_index = np.digitize(hist[i], bins) - 1
        if 0 <= bin_index < len(bin_counts):
            sum_of_weights[bin_index] += weights[i]
            sum_of_weights_squared[bin_index] += weights[i]**2
    return np.sqrt(sum_of_weights_squared)


dum_error = SumW2(bin_centers_dumdp, dumdp[cuts_dum], dum_scale, -10, 22, 100)
data_error = SumW2(bin_counts_datadp, ptardp[cuts_data], LD_scale, -10, 22, 100)
sub_error = SumW2(bin_counts_subdp, ptardp[cuts_data], LD_scale, -10, 22, 100)


plt.figure(figsize=(10, 6))

plt.hist(bin_centers_simdp, bins=len(bin_centers_simdp), histtype='step', weights=bin_counts_simdp, color='black', label="MC");
# plt.errorbar(bin_centers_simdp, bin_counts_simdp, fmt='+', label='MC')

plt.errorbar(bin_centers_dumdp, bin_counts_dumdp,yerr=dum_error,fmt='.', label="Dummy")
plt.errorbar(bin_centers_datadp, bin_counts_datadp,yerr=data_error,fmt='.', label='LD2')
plt.errorbar(bin_centers_subdp, bin_counts_subdp,yerr=sub_error,fmt='.', label='LD2 - dummy')

plt.title("Dummy subtraction (Delta)")
plt.xlabel("Delta")
plt.ylabel("Counts")
plt.yscale("log")
plt.legend()